In [ ]:
import pandas as pd
import requests
from abc import ABCMeta, abstractmethod
from bs4 import BeautifulSoup, NavigableString, Tag

# https://www.pref.miyazaki.lg.jp/kense/koho/kense-faq/index.html
# https://www.shinsei.pref.mie.lg.jp/www2/guide/faq.html
# https://www.pref.gunma.jp/03/x0110028.html
# https://www.pref.aomori.lg.jp/kenminno-koe/faq_matome.html


# 指定したURLからSoupを取得する
def getHtmltoSoup(url, proxy=None):
    res = requests.get(url, proxies=proxy)
    soup = BeautifulSoup(res.content, "html.parser")
    return soup

# Soupに対してCSSセレクタを指定してlistで返す
# TODO: あまり使わないかも.関数化する意味ないねぇ
def cssSelector(soup, selector) -> list:
    data_list = []
    for elems in soup.select(selector):
        data_list.append(elems)
    return data_list

# Component Class
class Content(metaclass=ABCMeta):
    @abstractmethod
    def getContents(self):
        pass

    #
    # private
    #
    # 改行コードを潰す
    def _scrubCrLf(self, str):
        return str.replace("\r\n", "").replace("\n", "")

# Composite Class
class SiteTree(Content):
    # @return SiteTree
    @staticmethod
    def create(siteParser, soup):
        category_list = siteParser.root2CategoryList(soup)
        return SiteTree(category_list)

    def __init__(self, category_list):
        self.category_list = category_list
 
    def getContents(self):
        return "\n".join(map(lambda x: x.getContents(), self.category_list))

# Composite Class
class Category(Content):
    # @return Category
    @staticmethod
    def create(siteParser, tag_list):
        title = siteParser.category2title(tag_list)
        question_list = siteParser.category2QuestionList(tag_list)
        return Category(title, question_list)

    def __init__(self, title, question_list):
        self.title = title
        self.question_list = question_list
 
    def getContents(self):
        return "\n".join(map(lambda x: self.title + "," + x.getContents(), self.question_list))

# Composite Class
class Question(Content):
    _id = 0

    @staticmethod
    def createUniqueId():
        Question._id += 1
        return Question._id

    # @return Question
    @staticmethod
    def create(siteParser, tag_list):
        id = Question.createUniqueId()
        title = siteParser.question2Title(tag_list)
        answer = siteParser.question2Answer(tag_list)
        return Question(id, title, title, answer)

    def __init__(self, id, title, body, answer):
        self.id = id 
        self.title = title
        self.body = body
        self.answer = answer

    def getContents(self):
        return ",".join([str(self.id),
                          self.title,
                          self.body,
                          self.answer.getContents()])

# Leaf Class
class Answer(Content):
    # @return Answer
    @staticmethod
    def create(siteParser, tag_list):
        body = siteParser.answer2Body(tag_list)
        return Answer(body)

    def __init__(self, body, comment='', state='approved', stateId='3'):
        self.body = self._scrubCrLf(body)
        self.comment = self._scrubCrLf(comment)
        self.state = state
        self.stateId = stateId

    def getContents(self):
        return ",".join([self.body,
                          self.comment,
                          self.state,
                          self.stateId])


class SiteParser(metaclass=ABCMeta):
    # @return String
    @abstractmethod
    def getRootURL(self):
        pass

    # @return List<Category>
    # param soup: ページ全体のSoup
    @abstractmethod
    def root2CategoryList(self, soup):
        pass

    # @return String
    # param tag_list: 1Category分のList<Tag>
    @abstractmethod
    def category2title(self, tag_list):
        pass

    # @return List<Question>
    # param tag_list: 1Category分のList<Tag>
    @abstractmethod
    def category2QuestionList(self, tag_list):
        pass

    # @return String
    # param tag_list: 1Question分のList<Tag>
    @abstractmethod
    def question2Title(self, tag_list):
        pass

    # @return Answer
    # param tag_list: 1Question分のList<Tag>
    @abstractmethod
    def question2Answer(self, tag_list):
        pass

    # @return String
    # param tag_list: 1Answer分のList<Tag>
    @abstractmethod
    def answer2Body(self, tag_list):
        pass
#
# 群馬県
#
class SiteGumma(SiteParser):
    # エンドポイント
    def getRootURL(self):
        return 'https://www.pref.gunma.jp/03/x0110028.html'
    #
    # root
    #
    def root2CategoryList(self, soup):
        tag_list = self._extractTargetTagList(soup)
        category_list, buf = [], []
        for t in tag_list:
            if t.name == "h3" and buf: # h3要素がcategoryのタイトル
                category_list.append(Category.create(self, buf))
                buf = []
            buf.append(t)
        if buf:
            category_list.append(Category.create(self, buf)) # 最後の要素をケア
        return category_list
    
    # ページ全体から必要な部分だけを取ってくる
    def _extractTargetTagList(self, soup):
        soup = soup.select('#scroll-main > .typeA > .honbun')
        tag_list = list(filter(lambda x: x.name != None, soup[0]))
        buf = []
        skip = True
        for t in tag_list:
            if not skip:
                buf.append(t)
            # よくある質問と回答まで読み飛ばす
            if skip and t.string and "よくある質問と回答" in t.string:
                skip = False
            if t.name == "hr" and buf: # ココまで
                break
        return buf

    #
    # Category
    #
    # @return String
    def category2title(self, tag_list):
        return tag_list[0].string

    # @return List<Question>
    def category2QuestionList(self, tag_list):
        del tag_list[0] # categoryのタイトルを飛ばす
        question_list, buf = [], []
        for t in tag_list:
            if t.name == "h4" and buf: # h4要素がquestionのタイトル
                question_list.append(Question.create(self, buf))
                buf = []
            buf.append(t)
        if buf: # 最後の要素をケア
            question_list.append(Question.create(self, buf))
        return question_list

    #
    # Question
    #
    # @return String
    def question2Title(self, tag_list):
        return tag_list[0].string

    # @return Answer
    def question2Answer(self, tag_list):
        del tag_list[0] # titleを飛ばす
        return Answer.create(self, tag_list)
    
    #
    # Answer
    #
    # @return String
    def answer2Body(self, tag_list):
        buf = []
        for t in tag_list:
            buf.append(self._tag2String(t))
        return "".join(buf)

    #
    # private
    #
    # タグの中身を再帰的に文字列化する
    def _tag2String(self, tag):
        if tag.string:
            return tag.string
        else:
            return " ".join(map(lambda x: self._tag2String(x), tag))

### ここからmain ###
if __name__ == '__main__':
    parser = SiteGumma() # サイトによってこれを変える

    soup = getHtmltoSoup(parser.getRootURL())
    tree = SiteTree.create(parser, soup)

    # print(tree.category_list[2].question_list[3]getContents())
    print(tree.getContents())



１　教員採用試験,1,１－１　教員採用試験を受験したいのですが、提出書類はどこで配付していますか。,１－１　教員採用試験を受験したいのですが、提出書類はどこで配付していますか。,（問い合わせ先電話番号）小中学校：学校人事課　027-226-4593、高等学校・特別支援学校：学校人事課　027-226-4597　教員採用試験の提出書類は、県内では学校人事課（ 群馬県庁 23階）、県民センター（ 群馬県庁 2階）、群馬県内の教育事務所、行政県税事務所で、県外では  群馬県東京事務所 、ぐんま暮らし支援センター、群馬県大阪事務所で配付しています。また、郵送も可能です。詳しくは、 「2020年度採用群馬県公立学校教員募集要項」 をご覧ください。,,approved,3
１　教員採用試験,2,１－２　教員採用試験の過去問題は閲覧できますか。,１－２　教員採用試験の過去問題は閲覧できますか。,（問い合わせ先電話番号）小中学校：学校人事課　027-226-4593、高等学校・特別支援学校：学校人事課　027-226-4597　過去3年分の問題については、県民センター（ 群馬県庁 2階）で閲覧することができます。また、総合教育センターのホームページ上（ 群馬県教育委員会【各課発行・提供資料】中の「学校人事課」 ：外部リンク）で公開しています。,,approved,3
１　教員採用試験,3,１－３　教員採用試験で、所有している免許状が専修、1種と2種では採用試験や働く上での違いがありますか。,１－３　教員採用試験で、所有している免許状が専修、1種と2種では採用試験や働く上での違いがありますか。,（問い合わせ先電話番号）小中学校：学校人事課　027-226-4593、高等学校・特別支援学校：学校人事課　027-226-4597　所有免許状の専修、1種、2種を区別して採用していませんので、2種免許状所有者であっても不利ということはありません。　なお、2種免許状を所有し、教員に採用された方（教育職員）は、相当の1種免許状を取得するよう努めなければなりません（教育職員免許法第9条の5）。,,approved,3
１　教員採用試験,4,１－４　私は日本国籍ではないのですが、教員として採用されますか。,１－４　私は日本国籍ではないのですが、教員として採用されますか。,（問い合わせ先電話番号）小中